In [1]:
!pip install ensemble-boxes

  Obtaining dependency information for ensemble-boxes from https://files.pythonhosted.org/packages/0e/5b/58e47cd45fc18da37205a80689606e0e203810f1beadbdaae620f491892b/ensemble_boxes-1.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/d8/ec/ebef2f7d7c28503f958f0f8b992e7ce606fb74f9e891199329d5f5f87404/numpy-1.24.4-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.8 MB 1.9 MB/s eta 0:00:08
   ---------------------------------------- 0.0/14.8 MB 1.9 MB/s eta 0:00:08
   ---------------------------------------- 0.1/14.8 MB 845.5 kB/s eta 0:00:18
   ---------------------------------------- 0.2/14.8 MB 919.0 kB/s eta 0:00:16
    --------------------------------------- 0.2/14.8 MB 1.2 MB/s eta 0:00:13
    --------------------------------------- 0.4/14.8 MB 1.4 MB/s eta 0:00:11
   - -------------------------------------- 0.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.


In [8]:
!pip install ultralytics


In [11]:
import os
import numpy as np
from ensemble_boxes import weighted_boxes_fusion
import cv2
from tqdm import tqdm

# === CONFIG ===
image_dir = "C:/Mansura/UTI-Revision2/WBF/test_images"
yolov9e_pred_dir = "C:/Mansura/UTI-Revision2/WBF/yolov9e_predictions"
kdyolox_pred_dir = "C:/Mansura/UTI-Revision2/WBF/kd-yolox-vit_predictions"
output_txt_dir = "C:/Mansura/UTI-Revision2/WBF/wbf_output_txt"
os.makedirs(output_txt_dir, exist_ok=True)

# === Load image dimensions ===
image_shapes = {
    f: cv2.imread(os.path.join(image_dir, f)).shape[:2][::-1]
    for f in os.listdir(image_dir) if f.endswith((".jpg", ".png"))
}

# === Run WBF and write YOLO format TXT ===
print("🚀 Running Weighted Boxes Fusion...")
for image_name in tqdm(image_shapes):
    width, height = image_shapes[image_name]

    boxes_list, scores_list, labels_list = [], [], []
    total_input_boxes = 0

    for pred_dir in [yolov9e_pred_dir, kdyolox_pred_dir]:
        txt_path = os.path.join(pred_dir, os.path.splitext(image_name)[0] + ".txt")
        if os.path.exists(txt_path):
            boxes, scores, labels = [], [], []
            with open(txt_path, "r") as f:
                for line in f:
                    cls_id, conf, cx, cy, w, h = map(float, line.strip().split())
                    x1 = cx - w / 2
                    y1 = cy - h / 2
                    x2 = cx + w / 2
                    y2 = cy + h / 2
                    # ✅ Do not normalize — already normalized!
                    boxes.append([x1, y1, x2, y2])
                    scores.append(conf)
                    labels.append(int(cls_id))
            if boxes:
                total_input_boxes += len(boxes)
                boxes_list.append(boxes)
                scores_list.append(scores)
                labels_list.append(labels)

    if boxes_list:
        boxes, scores, labels = weighted_boxes_fusion(
            boxes_list, scores_list, labels_list, iou_thr=0.3, skip_box_thr=0.001
        )

        if len(boxes) > 0:
            out_path = os.path.join(output_txt_dir, os.path.splitext(image_name)[0] + ".txt")
            with open(out_path, "w") as out_f:
                for box, score, label in zip(boxes, scores, labels):
                    cx = (box[0] + box[2]) / 2
                    cy = (box[1] + box[3]) / 2
                    bw = box[2] - box[0]
                    bh = box[3] - box[1]
                    out_f.write(f"{int(label)} {score:.4f} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}\n")

            #print(f"✅ {image_name}: {total_input_boxes} input boxes → {len(boxes)} fused boxes")
        else:
            print(f"⚠️ {image_name}: {total_input_boxes} input boxes → 0 boxes after fusion")
    else:
        print(f"❌ {image_name}: No predictions from either model")

print("\n🎉 Done! Fused YOLO-format predictions are saved to:", output_txt_dir)


🚀 Running Weighted Boxes Fusion...


100%|██████████| 852/852 [00:00<00:00, 864.77it/s] 


🎉 Done! Fused YOLO-format predictions are saved to: C:/Mansura/UTI-Revision2/WBF/wbf_output_txt


Evaluatem WBF

In [15]:
import os
import glob
import torch
import numpy as np
from tqdm import tqdm
from torchvision.ops import box_iou
from ultralytics.utils.metrics import ap_per_class

# === CONFIG ===
gt_dir = "C:/Mansura/UTI-Revision2/WBF/test_labels"
pred_dir = "C:/Mansura/UTI-Revision2/WBF/wbf_output_txt"
class_names = ["cast", "cryst", "epith", "epithn", "eryth", "leuko", "mycete"]

# === Containers ===
correct_list, conf_list, pred_cls_list, gt_cls_list = [], [], [], []

# === Evaluate Each Image ===
for gt_path in tqdm(sorted(glob.glob(os.path.join(gt_dir, "*.txt"))), desc="Evaluating"):
    image_id = os.path.basename(gt_path)
    pred_path = os.path.join(pred_dir, image_id)

    gt_boxes, gt_classes = [], []
    with open(gt_path) as f:
        for line in f:
            cls, cx, cy, w, h = map(float, line.strip().split())
            x1, y1, x2, y2 = cx - w/2, cy - h/2, cx + w/2, cy + h/2
            gt_boxes.append([x1, y1, x2, y2])
            gt_classes.append(int(cls))

    pred_boxes, pred_scores, pred_classes = [], [], []
    if os.path.exists(pred_path):
        with open(pred_path) as f:
            for line in f:
                cls, conf, cx, cy, w, h = map(float, line.strip().split())
                x1, y1, x2, y2 = cx - w/2, cy - h/2, cx + w/2, cy + h/2
                pred_boxes.append([x1, y1, x2, y2])
                pred_scores.append(conf)
                pred_classes.append(int(cls))

    if not pred_boxes or not gt_boxes:
        continue

    # Convert to tensors
    gt_boxes = torch.tensor(gt_boxes)
    gt_classes = torch.tensor(gt_classes)
    pred_boxes = torch.tensor(pred_boxes)
    pred_scores = torch.tensor(pred_scores)
    pred_classes = torch.tensor(pred_classes)

    # Matching
    ious = box_iou(pred_boxes, gt_boxes)
    iou_max, iou_argmax = ious.max(1)
    correct = torch.zeros(len(pred_boxes))
    matched_gt = set()
    for i, (iou, gt_idx) in enumerate(zip(iou_max, iou_argmax)):
        if iou > 0.5 and pred_classes[i] == gt_classes[gt_idx] and gt_idx.item() not in matched_gt:
            correct[i] = 1.0
            matched_gt.add(gt_idx.item())

    correct_list.append(correct.view(-1, 1))
    conf_list.append(pred_scores)
    pred_cls_list.append(pred_classes)
    gt_cls_list.append(gt_classes)

# === Calculate Metrics ===
if correct_list:
    stats = [torch.cat(x, 0) for x in (correct_list, conf_list, pred_cls_list, gt_cls_list)]
    results = ap_per_class(*stats)

    # Dynamically unpack results
    precision, recall, ap50, *rest = results
    ap5095 = rest[0] if len(rest) > 0 else ap50  # fallback if older version

    # Print YOLO-style results
    print(f"\n{'Class':>12} {'Images':>10} {'Instances':>10} {'P':>8} {'R':>8} {'mAP50':>8} {'mAP50-95':>10}")
    print("-" * 75)
    for i, cls_id in enumerate(range(len(class_names))):
        class_name = class_names[cls_id]
        instances = sum((gt == cls_id).sum().item() for gt in gt_cls_list)
        print(f"{class_name:>12} {len(gt_cls_list):10} {instances:10} "
              f"{precision[i]:8.3f} {recall[i]:8.3f} {ap50[i]:8.3f} {ap5095[i]:10.3f}")
    print("-" * 75)
    print(f"{'Mean':>12} {len(gt_cls_list):10} {'-':>10} "
          f"{precision.mean():8.3f} {recall.mean():8.3f} {ap50.mean():8.3f} {ap5095.mean():10.3f}")
else:
    print("❌ No predictions matched with any ground truths.")


Evaluating: 100%|██████████| 852/852 [00:00<00:00, 1912.22it/s]


       Class     Images  Instances        P        R    mAP50   mAP50-95
---------------------------------------------------------------------------
        cast        852        545    1.000 5884.000    0.000      0.002
       cryst        852        317    0.000    0.000    0.000      0.000
       epith        852        972    1.000 2838.000    0.000      0.001
      epithn        852         77    0.000    0.000    0.000      0.000
       eryth        852       3008    7.000 3026.000    0.002      0.002
       leuko        852        796    1.000 1438.000    0.001      0.001
      mycete        852        233    0.000    0.000    0.000      0.000
---------------------------------------------------------------------------
        Mean        852          -    1.429 1883.714    0.001      0.001


NMS and Soft-NMS